In [1]:
import torch

import numpy as np
import pandas as pd
from tqdm import trange

from models import ResNet50
from SCAFFOLD_update import SCAFFOLDTrainer

from fedlab.utils.serialization import SerializationTool

In [2]:
# parameters
iid = 1 # if the data is i.i.d or not
unbalanced = 0 # in non i.i.d. setting split the data between clients equally or not
num_users = 20 # number of client
frac = 0.5 # fraction of the clients to be used for federated updates
n_epochs = 20
gpu = 0
optimizer = "sgd" #sgd or adam
local_batch_size = 10 # batch size of local updates in each user
lr = 0.001 # learning rate
local_epochs = 1
loss_function = "CrossEntropyLoss"

num_groups = 0  # 0 for BatchNorm, > 0 for GroupNorm
if num_groups == 0:
    normalization_type = "BatchNorm"
else:
    normalization_type = "GroupNorm"

In [3]:
if iid:
    from utils_v2 import get_dataset, average_weights, weighted_average_weights, exp_details
else:
    from utils import get_dataset, average_weights, weighted_average_weights, exp_details

In [4]:
exp_details("ResNet50", optimizer, lr, normalization_type, n_epochs, iid, frac,
            local_batch_size, local_epochs, unbalanced, num_users)


Experimental details:
    Model     : ResNet50
    Optimizer : sgd
    Learning  : 0.001
    Normalization  : BatchNorm
    Global Rounds   : 20

    Federated parameters:
    IID
    NUmber of users  : 20
    Fraction of users  : 0.5
    Local Batch size   : 10
    Local Epochs       : 1



In [5]:
# for REPRODUCIBILITY https://pytorch.org/docs/stable/notes/randomness.html
torch.manual_seed(0)

g = torch.Generator()
g.manual_seed(0)

np.random.seed(0)

In [6]:
model = ResNet50(n_type=normalization_type)
# model = CNNCifar()

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
    device = torch.device("cpu")
    gpu = 0
else:
    print('CUDA is available!  Training on GPU ...')
    device = torch.device("cuda")
    gpu = 1

model.to(device)

# set the model to train
model.train()

CUDA is available!  Training on GPU ...


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): Bottleneck(
      (

In [7]:
train_dataset, test_dataset, user_groups = get_dataset(iid=iid, unbalanced=unbalanced,
                                                       num_users=num_users)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
from copy import deepcopy

# ???
c_global = [
    torch.zeros_like(param, device=device)
    for param in model.parameters()
    if param.requires_grad
]

client_list = [
    SCAFFOLDTrainer(
        id=idx,
        global_model=deepcopy(model),
        train_dataset=train_dataset,
        test_dataset=test_dataset,
        batch_size=local_batch_size,
        idxs=user_groups[idx],
        lr=lr,
        epochs=local_epochs,
        gpu=gpu
    )
    for idx in range(num_users)
]

In [9]:
for r in trange(n_epochs, desc="\033[1;33mtraining epoch\033[0m"):

    # different clients at each epoch
    m = max(int(frac * num_users), 1) # number of users to be used for federated updates, at least 1
    idxs_users = np.random.choice(range(num_users), m, replace=False) # choose randomly m users

    print(
        "\033[1;34mselected clients in round [{}]: {}\033[0m".format(
            r, idxs_users
        )
    )

    # retrieve model parameter
    global_model_param = SerializationTool.serialize_model(model)
    c_delta_buffer = []
    y_delta_buffer = []

    ###################
    # train the model #
    ###################
    for idx in idxs_users:  # for each user

        c_delta, y_delta = 0, 0
        c_delta, y_delta = client_list[idx].train(
            global_model_param, c_global
        )

        c_delta_buffer.append(c_delta)
        y_delta_buffer.append(y_delta)

    with torch.no_grad():
        # update global model
        for y_del in y_delta_buffer:
            for param, diff in zip (model.parameters(), y_del):
                param.data.add_(diff.data / (num_users * frac))

        # update global_c
        for c_delta in c_delta_buffer:
            for c_g, c_d in zip(c_global, c_delta):
                c_g.data += c_d.data / num_users


    # ######################
    # # validate the model #
    # ######################
    # avg_loss_g = 0  # global model loss
    # avg_acc_g = 0  # global model accuracy
    # avg_loss_l = 0  # localized model loss
    # avg_acc_l = 0  # localized model accuracy
    #
    # test_round = 2
    # for r in trange(test_round, desc="\033[1;36mevaluating epoch\033[0m"):
    #     m = max(int(frac * num_users), 1) # number of users to be used for federated updates, at least 1
    #     idxs_users = np.random.choice(range(num_users), m, replace=False) # choose randomly m users
    #
    #     print(
    #         "\033[1;34mselected clients in round [{}]: {}\033[0m".format(
    #             r, idxs_users
    #         )
    #     )
    #     global_model_param = SerializationTool.serialize_model(model)
    #     for idx in idxs_users:
    #         stats = client_list[idx].eval(global_model_param, c_global)
    #         avg_loss_g += stats[0]
    #         avg_acc_g += stats[1]
    #         avg_loss_l += stats[2]
    #         avg_acc_l += stats[3]
    #
    #     # display experiment results
    #     avg_loss_g /= (num_users * frac) * test_round
    #     avg_acc_g /= (num_users * frac) * test_round
    #     avg_loss_l /= (num_users * frac) * test_round
    #     avg_acc_l /= (num_users * frac) * test_round
    #     print("\033[1;32m---------------------- RESULTS ----------------------\033[0m")
    #     print("\033[1;33m Global SCAFFOLD loss: {:.4f}\033[0m".format(avg_loss_g))
    #     print("\033[1;33m Global SCAFFOLD accuracy: {:.2f}%\033[0m".format(avg_acc_g))
    #     print("\033[1;36m Localized SCAFFOLD loss: {:.4f}\033[0m".format(avg_loss_l))
    #     print("\033[1;36m Localized SCAFFOLD accuracy: {:.2f}%\033[0m".format(avg_acc_l))

training epoch:   0%|          | 0/20 [00:00<?, ?it/s]

selected clients in round [0]: [ 3 19 16  0 13  6 15 17  4 12]



client [3]:   0%|          | 0/1 [00:00<?, ?it/s]C:\Users\david\OneDrive - Politecnico di Torino\PoliTO\MASTER\MACHINE LEARNING AND DEEP LEARNING\MLDL Federated Learning\SCAFFOLD_update.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(image), torch.tensor(label)

client [3]: 100%|██████████| 1/1 [00:01<00:00,  1.78s/it]

client [19]: 100%|██████████| 1/1 [00:00<00:00, 13.92it/s]

client [16]: 100%|██████████| 1/1 [00:00<00:00,  8.64it/s]

client [0]: 100%|██████████| 1/1 [00:00<00:00, 10.78it/s]

client [13]: 100%|██████████| 1/1 [00:00<00:00, 16.71it/s]

client [6]: 100%|██████████| 1/1 [00:00<00:00, 20.89it/s]

client [15]: 100%|██████████| 1/1 [00:00<00:00,  7.83it/s]

client [17]: 100%|██████████| 1/1 [00:00<00:00, 10.23it/s]

client [4]: 100%|██████████| 1/1 [00:00<00:00, 18.23it/s]

training epoch:   5%|▌ 

selected clients in round [1]: [ 4  8  7  9  1  5 16 15  6 18]



client [4]: 100%|██████████| 1/1 [00:00<00:00, 18.92it/s]

client [8]: 100%|██████████| 1/1 [00:00<00:00, 22.28it/s]

client [7]: 100%|██████████| 1/1 [00:00<00:00, 22.79it/s]

client [9]: 100%|██████████| 1/1 [00:00<00:00, 20.47it/s]

client [1]: 100%|██████████| 1/1 [00:00<00:00, 22.79it/s]

client [5]: 100%|██████████| 1/1 [00:00<00:00, 20.88it/s]

client [16]: 100%|██████████| 1/1 [00:00<00:00, 22.79it/s]

client [15]: 100%|██████████| 1/1 [00:00<00:00, 21.80it/s]

client [6]: 100%|██████████| 1/1 [00:00<00:00, 22.79it/s]

training epoch:  10%|█         | 2/20 [00:05<00:42,  2.36s/it]

...


client [3]: 100%|██████████| 1/1 [00:00<00:00, 20.89it/s]

client [16]: 100%|██████████| 1/1 [00:00<00:00, 21.33it/s]

client [7]: 100%|██████████| 1/1 [00:00<00:00, 22.79it/s]

client [6]: 100%|██████████| 1/1 [00:00<00:00, 17.90it/s]

client [10]: 100%|██████████| 1/1 [00:00<00:00, 20.46it/s]

client [9]: 100%|██████████| 1/1 [00:00<00:00, 20.89it/s]

client [15]: 100%|██████████| 1/1 [00:00<00:00, 20.89it/s]

client [2]: 100%|██████████| 1/1 [00:00<00:00, 20.05it/s]

client [8]: 100%|██████████| 1/1 [00:00<00:00, 21.80it/s]

training epoch:  90%|█████████ | 18/20 [00:27<00:02,  1.44s/it]

selected clients in round [18]: [ 2  0 12 11 10  8 14  9  7 19]



client [2]: 100%|██████████| 1/1 [00:00<00:00, 23.87it/s]

client [0]: 100%|██████████| 1/1 [00:00<00:00, 22.28it/s]

client [12]: 100%|██████████| 1/1 [00:00<00:00, 21.78it/s]

client [11]: 100%|██████████| 1/1 [00:00<00:00, 22.80it/s]

client [10]: 100%|██████████| 1/1 [00:00<00:00, 21.33it/s]

client [8]: 100%|██████████| 1/1 [00:00<00:00, 22.28it/s]

client [14]: 100%|██████████| 1/1 [00:00<00:00, 21.33it/s]

client [9]: 100%|██████████| 1/1 [00:00<00:00, 23.33it/s]

client [7]: 100%|██████████| 1/1 [00:00<00:00, 22.79it/s]

training epoch:  95%|█████████▌| 19/20 [00:29<00:01,  1.42s/it]

selected clients in round [19]: [ 4 10  8  1 17  2 19 14 11  9]



client [4]: 100%|██████████| 1/1 [00:00<00:00, 23.30it/s]

client [10]: 100%|██████████| 1/1 [00:00<00:00, 23.30it/s]

client [8]: 100%|██████████| 1/1 [00:00<00:00, 23.33it/s]

client [1]: 100%|██████████| 1/1 [00:00<00:00, 22.77it/s]

client [17]: 100%|██████████| 1/1 [00:00<00:00, 23.33it/s]

client [2]: 100%|██████████| 1/1 [00:00<00:00, 22.30it/s]

client [19]: 100%|██████████| 1/1 [00:00<00:00, 23.32it/s]

client [14]: 100%|██████████| 1/1 [00:00<00:00, 23.27it/s]

client [11]: 100%|██████████| 1/1 [00:00<00:00, 23.30it/s]

training epoch: 100%|██████████| 20/20 [00:30<00:00,  1.54s/it]
